In [13]:
import requests
requests.get('http://localhost:9200').json()

{'name': 'k3DkNSo',
 'cluster_name': 'tubular',
 'cluster_uuid': 'ydZAhfywTsqMGGjZ5zTiGA',
 'version': {'number': '6.6.0',
  'build_flavor': 'default',
  'build_type': 'tar',
  'build_hash': 'a9861f4',
  'build_date': '2019-01-24T11:27:09.439740Z',
  'build_snapshot': False,
  'lucene_version': '7.6.0',
  'minimum_wire_compatibility_version': '5.6.0',
  'minimum_index_compatibility_version': '5.0.0'},
 'tagline': 'You Know, for Search'}

In [2]:
from elasticsearch import Elasticsearch
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

In [6]:
es.index(index='test-index', doc_type='test', id=1, body={'test': 'test'})
es.delete(index='test-index', doc_type='test', id=1)

{'_index': 'test-index',
 '_type': 'test',
 '_id': '1',
 '_version': 5,
 'result': 'deleted',
 '_shards': {'total': 2, 'successful': 1, 'failed': 0},
 '_seq_no': 4,
 '_primary_term': 1}

In [8]:
r = requests.get('http://localhost:9200') 
i = 1
while r.status_code == 200:
    r = requests.get(f'http://swapi.co/api/people/{i}')
    es.index(index='starwars', doc_type='people', id=i, body=r.json())
    i += 1

In [65]:
es.get(index='starwars', doc_type='people', id=1)['_source']['name']

'Luke Skywalker'

In [70]:
def search(query):
    return es.search(index='starwars', body={
        'query': {
            'multi_match': {
                'fields': ['name'],
                'query': query,
                'fuzziness': 2, # 'AUTO', #1,
                'operator': 'and'
            }
         }
    })

search('obi')

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0},
 'hits': {'total': 1,
  'max_score': 1.2067742,
  'hits': [{'_index': 'starwars',
    '_type': 'people',
    '_id': '10',
    '_score': 1.2067742,
    '_source': {'name': 'Obi-Wan Kenobi',
     'height': '182',
     'mass': '77',
     'hair_color': 'auburn, white',
     'skin_color': 'fair',
     'eye_color': 'blue-gray',
     'birth_year': '57BBY',
     'gender': 'male',
     'homeworld': 'https://swapi.co/api/planets/20/',
     'films': ['https://swapi.co/api/films/2/',
      'https://swapi.co/api/films/5/',
      'https://swapi.co/api/films/4/',
      'https://swapi.co/api/films/6/',
      'https://swapi.co/api/films/3/',
      'https://swapi.co/api/films/1/'],
     'species': ['https://swapi.co/api/species/1/'],
     'vehicles': ['https://swapi.co/api/vehicles/38/'],
     'starships': ['https://swapi.co/api/starships/48/',
      'https://swapi.co/api/starships/59/',
      'https:

In [ ]:
# https://www.elastic.co/guide/en/elasticsearch/reference/current/search-suggesters-completion.html
# https://github.com/elastic/elasticsearch-py/tree/master/example
# TODO(DAN): better stemmers/analyzers, perform index update